Turn this into a script

In [1]:
import laspy
import lazrs
from pathlib import Path
from shapely.ops import Point
import numpy as np
import rasterio
import geopandas as gpd

import rasterio
from rasterio.transform import from_origin
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
from numpy import int16, float32

import pandas as pd
from tqdm import tqdm

In [2]:
laz_fp = Path('D:/lidar_files')
export_fp = Path('D:/bridge_tif')
export_fp2 = Path('D:/bridge_points')

In [3]:
test = []

for fp in tqdm(list(laz_fp.glob('*.laz'))):
    
    las = laspy.read(fp)
    # 17 is the code for bridge decks
    x = np.array(las.x[las.classification == 17])
    y = np.array(las.y[las.classification == 17])
    z = np.array(las.z[las.classification == 17])
    geometry = [Point(x1,y1) for x1, y1 in zip(x,y)]
    geom_value = list(zip(geometry,z))

    if len(geom_value) == 0:
        continue

    test = test + geom_value

  0%|          | 0/1390 [00:00<?, ?it/s]

100%|██████████| 1390/1390 [09:58<00:00,  2.32it/s]


In [9]:
len(test)

7397694

In [8]:
gdf = gpd.GeoDataFrame(test,columns=['geometry','elevation_m'],geometry='geometry',crs=las.header.parse_crs())

In [11]:
gdf.to_crs('epsg:4326',inplace=True)

In [14]:
gdf

,geometry,elevation_m
0,POINT (-84.46475 33.62046),310.917
1,POINT (-84.46475 33.62046),310.808
2,POINT (-84.46475 33.62047),309.291
3,POINT (-84.46474 33.62046),307.006
4,POINT (-84.46474 33.62046),307.242
...,...,...
7397689,POINT (-84.18918 33.75984),258.450
7397690,POINT (-84.18918 33.75983),258.540
7397691,POINT (-84.18918 33.75983),258.540
7397692,POINT (-84.18917 33.75983),258.500


In [13]:
import pickle
with Path('D:/lidar_points.pkl').open('wb') as fh:
    pickle.dump(gdf,fh)

Trim off after this 

In [ ]:
    geom_value = list(zip(geometry,z))
    count_value = list(zip(geometry,np.ones_like(z)))

    if len(geom_value) == 0:
        continue

    # Example extent calculation based on geometry extents
    xmin, ymin, xmax, ymax = las.x.min(), las.y.min(), las.x.max(), las.y.max()

    # resolution in meters
    pixel_size = 15

    # Calculate the width and height of the raster
    width = int((xmax - xmin) / pixel_size)
    height = int((ymax - ymin) / pixel_size)

    # Create a transformation
    transform = from_origin(xmin, ymax, pixel_size, pixel_size)

    #rasterize data
    count_raster = features.rasterize(count_value,
                                    out_shape=(height,width),
                                    fill = 0,#np.nan,
                                    transform=transform,
                                    all_touched=True,
                                    merge_alg = rasterio.enums.MergeAlg.add,#MergeAlg.add,#.replace,
                                    dtype=float32)

    #rasterize data
    rasterized = features.rasterize(geom_value,
                                    out_shape=(height,width),
                                    fill = 0,#np.nan,
                                    transform=transform,
                                    all_touched=True,
                                    merge_alg = rasterio.enums.MergeAlg.add,#MergeAlg.add,#.replace,
                                    dtype=float32)

    #averaged
    averaged = rasterized / count_raster


    #create directory
    if export_fp.exists() == False:
        export_fp.mkdir()

    new_fp = fp.name.split('.laz')[0] + ".tif"

    with rasterio.open(
            export_fp / new_fp,
            "w",
            driver = "GTiff",
            crs = las.header.parse_crs(),
            transform = transform,
            dtype = rasterio.float32,
            count = 1,
            width = width,
            height = height) as dst:
        dst.write(averaged, indexes = 1)

In [ ]:
for fp in list(laz_fp.glob('*.laz')):
    
#fp = Path.cwd() / 'lidar_files/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1067n1254.laz'

    las = laspy.read(fp)
    # 17 is the code for bridge decks
    x = np.array(las.x[las.classification == 17])
    y = np.array(las.y[las.classification == 17])
    z = np.array(las.z[las.classification == 17])
    geometry = [Point(x1,y1) for x1, y1 in zip(x,y)]
    geom_value = list(zip(geometry,z))
    count_value = list(zip(geometry,np.ones_like(z)))

    if len(geom_value) == 0:
        continue

    # Example extent calculation based on geometry extents
    xmin, ymin, xmax, ymax = las.x.min(), las.y.min(), las.x.max(), las.y.max()

    # resolution in meters
    pixel_size = 15

    # Calculate the width and height of the raster
    width = int((xmax - xmin) / pixel_size)
    height = int((ymax - ymin) / pixel_size)

    # Create a transformation
    transform = from_origin(xmin, ymax, pixel_size, pixel_size)

    #rasterize data
    count_raster = features.rasterize(count_value,
                                    out_shape=(height,width),
                                    fill = 0,#np.nan,
                                    transform=transform,
                                    all_touched=True,
                                    merge_alg = rasterio.enums.MergeAlg.add,#MergeAlg.add,#.replace,
                                    dtype=float32)

    #rasterize data
    rasterized = features.rasterize(geom_value,
                                    out_shape=(height,width),
                                    fill = 0,#np.nan,
                                    transform=transform,
                                    all_touched=True,
                                    merge_alg = rasterio.enums.MergeAlg.add,#MergeAlg.add,#.replace,
                                    dtype=float32)

    #averaged
    averaged = rasterized / count_raster


    #create directory
    if export_fp.exists() == False:
        export_fp.mkdir()

    new_fp = fp.name.split('.laz')[0] + ".tif"

    with rasterio.open(
            export_fp / new_fp,
            "w",
            driver = "GTiff",
            crs = las.header.parse_crs(),
            transform = transform,
            dtype = rasterio.float32,
            count = 1,
            width = width,
            height = height) as dst:
        dst.write(averaged, indexes = 1)

In [ ]:
las = laspy.read(Path.cwd() / 'lidar_files/USGS_LPC_GA_Statewide_2018_B18_DRRA_e1067n1254.laz')
lasa

In [ ]:
x = np.array(las.x[las.classification == 17])
y = np.array(las.y[las.classification == 17])
z = np.array(las.z[las.classification == 17])
shapes = {
    'coordinates': list(zip(x,y)),
    'type': 'Point',
    'data': z
}

In [ ]:
geometry = [Point(x1,y1) for x1, y1 in zip(x,y)]
geometry

In [ ]:
gdf = gpd.GeoDataFrame({'elevation_m':z,'geometry':geometry},crs=las.header.parse_crs())
#gdf.to_crs('epsg:26916',inplace=True)

In [ ]:
import rasterio
from rasterio.transform import from_origin
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
from numpy import int16, float32

# Assuming 'shapes' is a list of geometries
# You need to determine the extent and other parameters based on your data

# Example extent calculation based on shapes
xmin, ymin, xmax, ymax = x.min(), y.min(), x.max(), y.max()

# Example pixel size (adjust according to your preference)
pixel_size = 1

# Calculate the width and height of the raster
width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)

# Create a transformation
transform = from_origin(xmin, ymax, pixel_size, pixel_size)

# Define the CRS (Coordinate Reference System)
#crs = 'EPSG:4326'  # Adjust according to your data CRS

# Create the src object
src = rasterio.open(
    'output_raster.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,  # Number of bands
    dtype='uint8',  # Data type
    crs=las.header.parse_crs(),
    transform=transform
)


In [ ]:
geom_value = list(zip(gdf.geometry,gdf['elevation_m']))

In [ ]:
rasterized = features.rasterize(geom_value,
                                out_shape=src.shape,
                                fill = np.nan,
                                transform=src.transform,
                                all_touched=True,
                                merge_alg = MergeAlg.replace,
                                dtype=float32)

In [ ]:
import matplotlib.pyplot as plt
# Plot raster
fig, ax = plt.subplots(1, figsize = (10, 10))
show(rasterized, ax = ax)
plt.gca().invert_yaxis()

In [ ]:
rasterized.max()

In [ ]:
with rasterio.open(
        Path.cwd() / "rasterized_vector.tif", "w",
        driver = "GTiff",
        crs = src.crs,
        transform = src.transform,
        dtype = rasterio.float32,
        count = 1,
        width = src.width,
        height = src.height) as dst:
    dst.write(rasterized, indexes = 1)